In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Polycystic_Ovary_Syndrome"
cohort = "GSE151158"

# Input paths
in_trait_dir = "../../input/GEO/Polycystic_Ovary_Syndrome"
in_cohort_dir = "../../input/GEO/Polycystic_Ovary_Syndrome/GSE151158"

# Output paths
out_data_file = "../../output/preprocess/Polycystic_Ovary_Syndrome/GSE151158.csv"
out_gene_data_file = "../../output/preprocess/Polycystic_Ovary_Syndrome/gene_data/GSE151158.csv"
out_clinical_data_file = "../../output/preprocess/Polycystic_Ovary_Syndrome/clinical_data/GSE151158.csv"
json_path = "../../output/preprocess/Polycystic_Ovary_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional analysis of non-fibrotic NAFLD progression"
!Series_summary	"Background & Aims:  Non-alcoholic steatohepatitis (NASH), a subtype of non-alcoholic fatty liver disease (NAFLD) that can lead to fibrosis, cirrhosis, and hepatocellular carcinoma, is characterized by hepatic inflammation.  Despite evolving therapies aimed to ameliorate inflammation in NASH, the transcriptional changes that lead to inflammation progression in NAFLD remain poorly understood.  The aim of this study is to define transcriptional changes in early, non-fibrotic NAFLD using a biopsy-proven non-fibrotic NAFLD cohort.  Methods:  We extracted RNA from liver tissue of 40 patients with biopsy-proven NAFLD based on NAFLD Activity Score (NAS) (23 with NAS ≤3, 17 with NAS ≥5) and 21 healthy controls and compared changes in expression of 594 genes involved in innate immune function.    Results:  Compared to healthy controls, NAFLD patients with NAS ≥5 had differential ex

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import re

# 1. Gene Expression Data Availability
# Based on the background information and series title, this appears to be transcriptional analysis
# which suggests gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For Polycystic Ovary Syndrome (PCOS), I found row 11 with 'pcos: N' values
trait_row = 11

# For age, I found row 1 with age values
age_row = 1

# For gender, I found row 2 with 'Sex: F' and 'Sex: M' values
gender_row = 2

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert PCOS trait values to binary (0 or 1)"""
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # PCOS: Y would be 1, PCOS: N would be 0
    if value.upper() == 'Y':
        return 1
    elif value.upper() == 'N':
        return 0
    else:
        return None

def convert_age(value):
    """Convert age to a continuous value"""
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None

# Initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the clinical data
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        preview = preview_df(clinical_features)
        print("Clinical Features Preview:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save as CSV
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to: {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Polycystic_Ovary_Syndrome/GSE151158/GSE151158_series_matrix.txt.gz
Gene data shape: (618, 66)
First 20 gene/probe identifiers:
Index(['ABCB1', 'ABCF1', 'ABL1', 'ADA', 'AHR', 'AICDA', 'AIRE', 'ALAS1', 'APP',
       'AREG', 'ARG1', 'ARG2', 'ARHGDIB', 'ATG10', 'ATG12', 'ATG16L1', 'ATG5',
       'ATG7', 'ATM', 'B2M'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (ABCB1, ABCF1, etc.) are indeed standard human gene symbols.
# No additional mapping is required as they are already in the proper format for human genes.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. First, extract and save the clinical data since it's missing
# Get the SOFT and matrix file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get the background info and clinical data again
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Define the conversion functions from Step 2
def convert_trait(value):
    """Convert PCOS trait to binary (0 = control, 1 = PCOS)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if 'PCOS' in value:
        return 1
    else:
        return 0

def convert_gender(value):
    """Convert gender to binary (0 = female, 1 = male)
    Note: In this context, we're dealing with biological sex rather than gender identity
    Female-to-male transsexuals are biologically female (0)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Female is 0, Male is 1
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower() and 'female to male' not in value.lower():
        return 1
    else:
        return 0  # Female to male transsexuals are recorded as female (0) biologically

# Extract clinical features with the correct row indices from previous steps
trait_row = 1  # Contains "disease state: PCOS"
gender_row = 0  # Contains gender information
age_row = None  # Age information is not available in this dataset

# Process and save clinical data
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=None,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to: {out_clinical_data_file}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 2. Normalize gene symbols using synonym information from NCBI
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene identifiers:")
print(gene_data.index[:10])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
# First check how many samples have missing trait values
if trait in linked_data.columns:
    missing_trait = linked_data[trait].isna().sum()
    print(f"Samples with missing trait values: {missing_trait} out of {len(linked_data)}")

# Check gene missing value percentages
gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
gene_missing_pct = linked_data[gene_cols].isna().mean()
genes_to_keep = gene_missing_pct[gene_missing_pct <= 0.2].index
print(f"Genes with ≤20% missing values: {len(genes_to_keep)} out of {len(gene_cols)}")

# Check sample missing value percentages
if len(gene_cols) > 0:
    sample_missing_pct = linked_data[gene_cols].isna().mean(axis=1)
    samples_to_keep = sample_missing_pct[sample_missing_pct <= 0.05].index
    print(f"Samples with ≤5% missing gene values: {len(samples_to_keep)} out of {len(linked_data)}")

# Apply missing value handling
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
trait_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "This dataset contains gene expression data from ovary biopsies of women with PCOS and female-to-male transsexual individuals, focusing on LH-induced gene expression."
is_gene_available = len(gene_data) > 0
is_trait_available = trait in linked_data.columns
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=trait_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable and linked_data_clean.shape[0] > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Clinical data saved to: ../../output/preprocess/Polycystic_Ovary_Syndrome/clinical_data/GSE151158.csv
Clinical data preview:
{'GSM4567420': [0.0, 0.0], 'GSM4567421': [0.0, 0.0], 'GSM4567422': [0.0, 0.0], 'GSM4567423': [0.0, 0.0], 'GSM4567424': [0.0, 0.0], 'GSM4567425': [0.0, 0.0], 'GSM4567426': [0.0, 0.0], 'GSM4567427': [0.0, 0.0], 'GSM4567428': [0.0, 0.0], 'GSM4567429': [0.0, 0.0], 'GSM4567430': [0.0, 0.0], 'GSM4567431': [0.0, 0.0], 'GSM4567432': [0.0, 0.0], 'GSM4567433': [0.0, 0.0], 'GSM4567434': [0.0, 0.0], 'GSM4567435': [0.0, 0.0], 'GSM4567436': [0.0, 0.0], 'GSM4567437': [0.0, 0.0], 'GSM4567438': [0.0, 0.0], 'GSM4567439': [0.0, 0.0], 'GSM4567440': [0.0, 0.0], 'GSM4567441': [0.0, 0.0], 'GSM4567442': [0.0, 0.0], 'GSM4567443': [0.0, 0.0], 'GSM4567444': [0.0, 0.0], 'GSM4567445': [0.0, 0.0], 'GSM4567446': [0.0, 0.0], 'GSM4567447': [0.0, 0.0], 'GSM4567448': [0.0, 0.0], 'GSM4567449': [0.0, 0.0], 'GSM4567450': [0.0, 0.0], 'GSM4567451': [0.0, 0.0], 'GSM4567452': [0.0, 0.0], 'GSM4567453': [0

A new JSON file was created at: ../../output/preprocess/Polycystic_Ovary_Syndrome/cohort_info.json
Dataset deemed not usable for associative studies. Linked data not saved.
